In [1]:
import pandas as pd
import datetime as dt
import urllib
import zipfile
import re

## DOWNLOAD AND UNZIP DATA

In [4]:
url = 'http://web.mta.info/developers/data/nyct/subway/google_transit.zip'
urllib.urlretrieve (url, "GTFS_nyc_Subway.zip")

('GTFS_nyc_Subway.zip', <httplib.HTTPMessage instance at 0x10c443170>)

In [6]:
zip_ref = zipfile.ZipFile("GTFS_nyc_Subway.zip", 'r')
zip_ref.extractall("GTFS_nyc_Subway")
zip_ref.close()

## GET STOPS SEQUENCE ALONG EACH LINE

In [3]:
# A20170625SUN_001150_7..S97R
## A20170625SUN means the service by Sunday Schedule
## 001150 is id for the trip, form the 1st stop to last stop, next round would be different id
## 7..S97R means 7 train would run South bound,
## stop at S97R route ( set of stops of local train and express train are different)
stop_times = pd.read_csv('GTFS_nyc_Subway/stop_times.txt')

stop_times['service_id']=stop_times['trip_id'].apply(lambda x: x.split('_')[0])
stop_times['sub_trip_id']=stop_times['trip_id'].apply(lambda x: x.split('_')[1])
stop_times['train+direction']=stop_times['trip_id'].apply(lambda x: x.split('_')[2])
del stop_times['stop_headsign']
del stop_times['pickup_type']
del stop_times['drop_off_type']
del stop_times['shape_dist_traveled']
stop_times['train'] = stop_times['train+direction'].apply(lambda x: x.split('.')[0])
stop_times['day'] = stop_times['service_id'].apply(lambda x: x[-3:])

In [4]:
stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,service_id,sub_trip_id,train+direction,train,day
0,A20170625SUN_001150_7..S97R,00:11:30,00:11:30,701S,1,A20170625SUN,001150,7..S97R,7,SUN
1,A20170625SUN_001150_7..S97R,00:14:00,00:14:00,702S,2,A20170625SUN,001150,7..S97R,7,SUN
2,A20170625SUN_001150_7..S97R,00:15:30,00:15:30,705S,3,A20170625SUN,001150,7..S97R,7,SUN
3,A20170625SUN_001150_7..S97R,00:16:30,00:16:30,706S,4,A20170625SUN,001150,7..S97R,7,SUN
4,A20170625SUN_001150_7..S97R,00:17:30,00:17:30,707S,5,A20170625SUN,001150,7..S97R,7,SUN


### 1)find the route,when the train stops at every station along the route

In [5]:
def find_the_longest(line):
    """
    Find the longest route of a certain line in weekday,
    return how many stops the longest routes have,
    and route id, service id
    """
    
    North = filter( lambda x: x[3]=='N', set(stop_times['train+direction']))
    South = filter( lambda x: x[3]=='S', set(stop_times['train+direction']))
    ### some train longest route of south bound and north bound are different, 
    ### eg. A train, North bound max 59 stops, South bound max 58 stops.
    
    ## North bound:
    N_length=stop_times[(stop_times['train']==line)&
                       (stop_times['day']=='WKD')&
                       (stop_times['train+direction'].isin(North))]['stop_sequence'].max()
    N_train_direction = list(set(stop_times[(stop_times['train']==line)&
                       (stop_times['day']=='WKD')&
                       (stop_times['train+direction'].isin(North))&
                       (stop_times['stop_sequence']==N_length)]['train+direction']))[0]
    ## South bound:
    S_length=stop_times[(stop_times['train']==line)&
                       (stop_times['day']=='WKD')&
                       (stop_times['train+direction'].isin(South))]['stop_sequence'].max()
    S_train_direction = list(set(stop_times[(stop_times['train']==line)&
                       (stop_times['day']=='WKD')&
                       (stop_times['train+direction'].isin(South))&
                       (stop_times['stop_sequence']==S_length)]['train+direction']))[0]
    
    service_id = list(set(stop_times[(stop_times['train']==line)&
                      (stop_times['day']=='WKD')&
                      (stop_times['stop_sequence']==N_length)]['service_id']))[0]
    return N_length, N_train_direction, S_length, S_train_direction, service_id

In [6]:
train={}
for i in set(stop_times['train']):
    train[i] = find_the_longest(i)
train

### H is A,S train in JFK,
### GS is Time Sq-42 St to Grand Central,
### FS is near brooklyn prospect park

{'1': (37, '1..N03R', 37, '1..S03R', 'A20170625WKD'),
 '2': (61, '2..N08R', 61, '2..S08R', 'A20170625WKD'),
 '3': (34, '3..N01R', 34, '3..S01R', 'A20170625WKD'),
 '4': (54, '4..N13R', 54, '4..S13R', 'A20170625WKD'),
 '5': (39, '5..N60R', 36, '5..S03R', 'A20170625WKD'),
 '6': (38, '6..N01R', 38, '6..S01R', 'A20170625WKD'),
 '7': (22, '7..N97R', 22, '7..S97R', 'A20170625WKD'),
 'A': (59, 'A..N09R', 58, 'A..S74R', 'B20170625WKD'),
 'B': (37, 'B..N45R', 37, 'B..S45R', 'B20170625WKD'),
 'C': (40, 'C..N04R', 40, 'C..S04R', 'B20170625WKD'),
 'D': (41, 'D..N05R', 41, 'D..S05R', 'B20170625WKD'),
 'E': (32, 'E..N05R', 32, 'E..S04R', 'B20170625WKD'),
 'F': (45, 'F..N69R', 45, 'F..S69R', 'B20170625WKD'),
 'FS': (4, 'FS.N01R', 4, 'FS.S01R', 'B20170625WKD'),
 'G': (21, 'G..N14R', 21, 'G..S14R', 'B20170625WKD'),
 'GS': (2, 'GS.N01R', 2, 'GS.S03R', 'A20170625WKD'),
 'H': (5, 'H..N21R', 5, 'H..S21R', 'B20170625WKD'),
 'J': (30, 'J..N12R', 30, 'J..S12R', 'B20170625WKD'),
 'L': (24, 'L..N01R', 24, 'L..S0

In [26]:
df_1= pd.DataFrame(columns=['train','bound','stop_sequence','stop_id',
                            'service_id','route_id'])
for i in train.keys():
#for i in ['1']:
    stops_1 = train[i][0]
    direction_1= train[i][1]
    stops_2 = train[i][2]
    direction_2= train[i][3]
    service_id = train[i][4]
    
    df_2= stop_times[(stop_times['train+direction'] == direction_1)&
            (stop_times['train']==i)&
            (stop_times['service_id']==service_id )].iloc[:stops_1][['stop_id','stop_sequence']]
    df_2['train'] = i
    df_2['bound'] = direction_1[3]
    df_2['service_id'] = service_id
    df_2['route_id'] = direction_1
    df_1 = pd.concat([df_1,df_2])
    
    
    df_2= stop_times[(stop_times['train+direction'] == direction_2)&
            (stop_times['train']==i)&
            (stop_times['service_id']==service_id )].iloc[:stops_2][['stop_id','stop_sequence']]
    df_2['train'] = i
    df_2['bound'] = direction_2[3]
    df_2['service_id'] = service_id
    df_2['route_id'] = direction_2
    df_1 = pd.concat([df_1,df_2])
    

In [29]:
df_1.head(3)

,bound,route_id,service_id,stop_id,stop_sequence,train
459700,N,FS.N01R,B20170625WKD,D26N,1,FS
459701,N,FS.N01R,B20170625WKD,S04N,2,FS
459702,N,FS.N01R,B20170625WKD,S03N,3,FS


In [30]:
df_1 = df_1.sort_values(['train','bound','stop_sequence'])
df_1 = df_1.reset_index(drop= True)
stops = pd.read_csv('GTFS_nyc_Subway/stops.txt')
df_1 = df_1.merge(stops[['stop_id','stop_name','stop_lat','stop_lon']],on=['stop_id'],how='left')

In [31]:
## stop_id, 139, 139N, 139S are all the same stop, 
## N and S just indicate different track directions
df_1.head(3)

,bound,route_id,service_id,stop_id,stop_sequence,train,stop_name,stop_lat,stop_lon
0,N,1..N03R,A20170625WKD,139N,1,1,Rector St,40.707513,-74.013783
1,N,1..N03R,A20170625WKD,138N,2,1,Cortlandt St,40.711835,-74.012188
2,N,1..N03R,A20170625WKD,137N,3,1,Chambers St,40.715478,-74.009266


In [32]:
df_1.to_csv('cleaned_data/subway_stops_sequence_weekday(longest_route)')

In [99]:
stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,service_id,sub_trip_id,train+direction,train,day
0,A20170625SUN_001150_7..S97R,00:11:30,00:11:30,701S,1,A20170625SUN,001150,7..S97R,7,SUN
1,A20170625SUN_001150_7..S97R,00:14:00,00:14:00,702S,2,A20170625SUN,001150,7..S97R,7,SUN
2,A20170625SUN_001150_7..S97R,00:15:30,00:15:30,705S,3,A20170625SUN,001150,7..S97R,7,SUN
3,A20170625SUN_001150_7..S97R,00:16:30,00:16:30,706S,4,A20170625SUN,001150,7..S97R,7,SUN
4,A20170625SUN_001150_7..S97R,00:17:30,00:17:30,707S,5,A20170625SUN,001150,7..S97R,7,SUN


In [100]:
df_2 = stop_times[['stop_id','train']]
df_2['stop_id'] =  [x[:-1] for x in df_2['stop_id']]
df_2 = df_2.drop_duplicates().sort_values(['stop_id','train']).reset_index(drop=True)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [106]:
df_2 = df_2.merge(stops[['stop_id','stop_name','stop_lat','stop_lon']],on='stop_id',how='left')

In [107]:
df_2.head()

,stop_id,train,stop_name,stop_lat,stop_lon
0,101,1,Van Cortlandt Park - 242 St,40.889248,-73.898583
1,103,1,238 St,40.884667,-73.900870
2,104,1,231 St,40.878856,-73.904834
3,106,1,Marble Hill - 225 St,40.874561,-73.909831
4,107,1,215 St,40.869444,-73.915279


In [108]:
df_2.to_csv('cleaned_data/which_lines_at_each_station(longest_route)')

In [110]:
no_service_stops = []
for i in stops['stop_id'][::3]:
    if i in df_2['stop_id'].values:
        pass
    else:
        no_service_stops.append(i)
print "recently no service at these stations"
stops[stops['stop_id'].isin(no_service_stops)][['stop_id','stop_name']]

recently no service at these stations


,stop_id,stop_name
111,140,South Ferry Loop
1221,M09,Knickerbocker Av
1224,M10,Central Av
1287,N12,S.B. Coney Island
1431,S10,Atlantic
1437,S12,Nassau


### 2）find the route for rush hour

In [14]:
stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,service_id,sub_trip_id,train+direction,train,day
0,A20170625SUN_001150_7..S97R,00:11:30,00:11:30,701S,1,A20170625SUN,001150,7..S97R,7,SUN
1,A20170625SUN_001150_7..S97R,00:14:00,00:14:00,702S,2,A20170625SUN,001150,7..S97R,7,SUN
2,A20170625SUN_001150_7..S97R,00:15:30,00:15:30,705S,3,A20170625SUN,001150,7..S97R,7,SUN
3,A20170625SUN_001150_7..S97R,00:16:30,00:16:30,706S,4,A20170625SUN,001150,7..S97R,7,SUN
4,A20170625SUN_001150_7..S97R,00:17:30,00:17:30,707S,5,A20170625SUN,001150,7..S97R,7,SUN


In [15]:
stop_times = stop_times[stop_times['day']=='WKD']
morning_rush_trip_id = stop_times[(stop_times['stop_sequence']==1)&
                                   stop_times['arrival_time'].isin(
                        filter(lambda x: (int(x[:2])>=7)&(int(x[:2])<10),stop_times['arrival_time']))]['sub_trip_id']

In [16]:
morning_rush = stop_times[stop_times['sub_trip_id'].isin(morning_rush_trip_id)]

In [17]:
morning_rush.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,service_id,sub_trip_id,train+direction,train,day
7612,A20170625WKD_045000_GS.N04R,07:30:00,07:30:00,901N,1,A20170625WKD,045000,GS.N04R,GS,WKD
7613,A20170625WKD_045000_GS.N04R,07:31:30,07:31:30,902N,2,A20170625WKD,045000,GS.N04R,GS,WKD
7614,A20170625WKD_045350_GS.S04R,07:33:30,07:33:30,902S,1,A20170625WKD,045350,GS.S04R,GS,WKD
7615,A20170625WKD_045350_GS.S04R,07:35:00,07:35:00,901S,2,A20170625WKD,045350,GS.S04R,GS,WKD
7616,A20170625WKD_045900_GS.N04R,07:39:00,07:39:00,901N,1,A20170625WKD,045900,GS.N04R,GS,WKD


In [18]:
morning_rush['arrival_time'] = pd.to_datetime(morning_rush['arrival_time'])
morning_rush.head(2)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,service_id,sub_trip_id,train+direction,train,day
7612,A20170625WKD_045000_GS.N04R,2017-06-29 07:30:00,07:30:00,901N,1,A20170625WKD,045000,GS.N04R,GS,WKD
7613,A20170625WKD_045000_GS.N04R,2017-06-29 07:31:30,07:31:30,902N,2,A20170625WKD,045000,GS.N04R,GS,WKD


In [19]:
start = morning_rush[morning_rush['stop_sequence']==1]

In [20]:
start.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,service_id,sub_trip_id,train+direction,train,day
7612,A20170625WKD_045000_GS.N04R,2017-06-29 07:30:00,07:30:00,901N,1,A20170625WKD,045000,GS.N04R,GS,WKD
7614,A20170625WKD_045350_GS.S04R,2017-06-29 07:33:30,07:33:30,902S,1,A20170625WKD,045350,GS.S04R,GS,WKD
7616,A20170625WKD_045900_GS.N04R,2017-06-29 07:39:00,07:39:00,901N,1,A20170625WKD,045900,GS.N04R,GS,WKD
7618,A20170625WKD_046250_GS.S04R,2017-06-29 07:42:30,07:42:30,902S,1,A20170625WKD,046250,GS.S04R,GS,WKD
7620,A20170625WKD_046800_GS.N04R,2017-06-29 07:48:00,07:48:00,901N,1,A20170625WKD,046800,GS.N04R,GS,WKD


In [473]:
### morning rush hour route as below, and how many trips are running at the route
start['train+direction'].value_counts()[:10]

7..N97R    50
L..S01R    40
L..N01R    36
E..N66R    35
E..S71R    34
7..S98R    34
1..N03R    33
1..S03R    31
4..N06R    28
6..N01R    27
Name: train+direction, dtype: int64

In [21]:
morning_rush_route =  list(set(morning_rush['train+direction'].values))

In [22]:
route_len = {}
for route in morning_rush_route:
    if route_len.get(route)==None:
        route_len[route]= morning_rush[morning_rush['train+direction'] == route]['stop_sequence'].max()
    else:
        pass

In [23]:
route_len.items()[:10]

[('B..N47R', 27),
 ('2..N01R', 49),
 ('F..S68R', 40),
 ('H..S21R', 5),
 ('N..S70R', 44),
 ('J..S21R', 23),
 ('L..S01R', 24),
 ('GS.N04R', 2),
 ('6..S02R', 29),
 ('A..S55X009', 36)]

In [525]:
# duration=pd.DataFrame(columns=['from_stop_id','to_stop_id','duration','route'])
# for i in route_len.keys()[:1]:
#     df = morning_rush[morning_rush['train+direction']==i].iloc[:route_len[i]]
#     duration['from_stop_id'] = df['stop_id'][:-1].values
#     duration['to_stop_id'] = df['stop_id'][1:].values
#     duration['duration']=map(dt.timedelta.total_seconds,(df['arrival_time'][1:].values-df['arrival_time'][:-1]))
#     duration['route'] = df['train+direction'].values[:-1]

In [24]:
duration=pd.DataFrame(columns=['from_stop_id','to_stop_id','duration','route'])
for i in route_len.keys():
    df = morning_rush[morning_rush['train+direction']==i].iloc[:route_len[i]]
    duration_1=pd.DataFrame(columns=['from_stop_id','to_stop_id','duration'])
    duration_1['from_stop_id'] = df['stop_id'][:-1].values
    duration_1['to_stop_id'] = df['stop_id'][1:].values
    duration_1['duration']=map(dt.timedelta.total_seconds,(df['arrival_time'][1:].values-df['arrival_time'][:-1]))
    duration_1['route'] = df['train+direction'].values[:-1]
    duration = pd.concat([duration,duration_1])

In [25]:
duration['train'] = map(lambda x:x.split('.')[0],duration['route'])
duration = pd.DataFrame(duration.groupby(['from_stop_id',
                               'to_stop_id',
                               'train'])['duration'].mean()).reset_index()

In [27]:
duration.head()

,from_stop_id,to_stop_id,train,duration
0,101S,103S,1,90.0
1,103N,101N,1,90.0
2,103S,104S,1,90.0
3,104N,103N,1,90.0
4,104S,106S,1,90.0


In [26]:
len(duration)

1493

#### there 1474 connections between stops at weekday morning rush hour

In [537]:
duration = duration.merge(stops.rename(columns={'stop_id':'from_stop_id',
                    "stop_name":'from_stop_name',
                     "stop_lat":'from_stop_lat',
                     "stop_lon":'from_stop_lon'})[['from_stop_id','from_stop_name','from_stop_lat','from_stop_lon']],
              on = 'from_stop_id',how = 'left')

In [538]:
duration = duration.merge(stops.rename(columns={'stop_id':'to_stop_id',
                    "stop_name":'to_stop_name',
                     "stop_lat":'to_stop_lat',
                     "stop_lon":'to_stop_lon'})[['to_stop_id','to_stop_name','to_stop_lat','to_stop_lon']],
              on = 'to_stop_id',how = 'left')

In [539]:
duration.head()

,from_stop_id,to_stop_id,train,duration,from_stop_name,from_stop_lat,from_stop_lon,to_stop_name,to_stop_lat,to_stop_lon
0,101S,103S,1,90.0,Van Cortlandt Park - 242 St,40.889248,-73.898583,238 St,40.884667,-73.900870
1,103N,101N,1,90.0,238 St,40.884667,-73.900870,Van Cortlandt Park - 242 St,40.889248,-73.898583
2,103S,104S,1,90.0,238 St,40.884667,-73.900870,231 St,40.878856,-73.904834
3,104N,103N,1,90.0,231 St,40.878856,-73.904834,238 St,40.884667,-73.900870
4,104S,106S,1,90.0,231 St,40.878856,-73.904834,Marble Hill - 225 St,40.874561,-73.909831


In [37]:
duration.to_csv('cleaned_data/duration_between_stops(morning_rush_hour)')

## FUNCTION FOR FINDING SCHEDULE OF ANY TIME ANY DAY

In [12]:
def find_duration(time1,time2,day='WKD'):
    """
    This funtion is used to get time duration between stops 
    based on different schedule of time of day.
    
    (time1,time2) defines train departure time at first stop between time1 and time 2
    time1 = 0,1,2,...,23
    time2 = 0,1,2,...,23
    eg. time1=7,time2=10, then train departure between 7:00:00 and 9:59:59 count
    day = 'WKD', 'SAT' or 'SUN'
    """
    ## read stop_times data and prepare it
    stop_times = pd.read_csv('GTFS_nyc_Subway/stop_times.txt')

    stop_times['service_id']=stop_times['trip_id'].apply(lambda x: x.split('_')[0])
    stop_times['sub_trip_id']=stop_times['trip_id'].apply(lambda x: x.split('_')[1])
    stop_times['train+direction']=stop_times['trip_id'].apply(lambda x: x.split('_')[2])
    del stop_times['stop_headsign']
    del stop_times['pickup_type']
    del stop_times['drop_off_type']
    del stop_times['shape_dist_traveled']
    stop_times['train'] = stop_times['train+direction'].apply(lambda x: x.split('.')[0])
    stop_times['day'] = stop_times['service_id'].apply(lambda x: x[-3:])
    
    stop_times = stop_times[stop_times['day']==day]
    
    
    ## get all the sub_trip_id departures from the 1st stops between time1 and time2
    morning_rush_trip_id = stop_times[(stop_times['stop_sequence']==1)&
                                   stop_times['arrival_time'].isin(
                                        filter(lambda x: (int(x[:2])>=time1)&
                                       (int(x[:2])<time2),stop_times['arrival_time']))]['sub_trip_id']
    
    ## get all the schedule of all sub_trip 
    morning_rush = stop_times[stop_times['sub_trip_id'].isin(morning_rush_trip_id)]
    morning_rush['arrival_time'] = pd.to_datetime(morning_rush['arrival_time'])
    
    ## get all the route id within the time period
    start = morning_rush[morning_rush['stop_sequence']==1]
    morning_rush_route =  list(set(morning_rush['train+direction'].values))
    
    ## get length of all the route, for next step purpose-- get the sequence of the route
    route_len = {}
    for route in morning_rush_route:
        if route_len.get(route)==None:
            route_len[route]= morning_rush[morning_rush['train+direction'] == route]['stop_sequence'].max()
        else:
            pass
    
    ## get schedule of all the routes run within the time period
    duration=pd.DataFrame(columns=['from_stop_id','to_stop_id','duration','route'])
    for i in route_len.keys():
        df = morning_rush[morning_rush['train+direction']==i].iloc[:route_len[i]]
        duration_1=pd.DataFrame(columns=['from_stop_id','to_stop_id','duration'])
        duration_1['from_stop_id'] = df['stop_id'][:-1].values
        duration_1['to_stop_id'] = df['stop_id'][1:].values
        duration_1['duration']=map(dt.timedelta.total_seconds,(df['arrival_time'][1:].values-df['arrival_time'][:-1]))
        duration_1['route'] = df['train+direction'].values[:-1]
        duration = pd.concat([duration,duration_1])
        
    ## occasionally, duration between two stops by same train is different from different route
    ## so I use mean to replace all the duplicates
    duration['train'] = map(lambda x:x.split('.')[0],duration['route'])
    duration = pd.DataFrame(duration.groupby(['from_stop_id',
                               'to_stop_id',
                               'train'])['duration'].mean()).reset_index()
    
    ## read stops data and merge stops name and geo info with duration dataframe
    stops = pd.read_csv('GTFS_nyc_Subway/stops.txt')
    
    duration = duration.merge(stops.rename(columns={'stop_id':'from_stop_id',
                    "stop_name":'from_stop_name',
                     "stop_lat":'from_stop_lat',
                     "stop_lon":'from_stop_lon'})[['from_stop_id','from_stop_name','from_stop_lat','from_stop_lon']],
              on = 'from_stop_id',how = 'left')
    duration = duration.merge(stops.rename(columns={'stop_id':'to_stop_id',
                    "stop_name":'to_stop_name',
                     "stop_lat":'to_stop_lat',
                     "stop_lon":'to_stop_lon'})[['to_stop_id','to_stop_name','to_stop_lat','to_stop_lon']],
              on = 'to_stop_id',how = 'left')
    return duration

In [38]:
duration = find_duration(7,10)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [36]:
duration.head()

,from_stop_id,to_stop_id,train,duration,from_stop_name,from_stop_lat,from_stop_lon,to_stop_name,to_stop_lat,to_stop_lon
0,101S,103S,1,90.0,Van Cortlandt Park - 242 St,40.889248,-73.898583,238 St,40.884667,-73.900870
1,103N,101N,1,90.0,238 St,40.884667,-73.900870,Van Cortlandt Park - 242 St,40.889248,-73.898583
2,103S,104S,1,90.0,238 St,40.884667,-73.900870,231 St,40.878856,-73.904834
3,104N,103N,1,90.0,231 St,40.878856,-73.904834,238 St,40.884667,-73.900870
4,104S,106S,1,90.0,231 St,40.878856,-73.904834,Marble Hill - 225 St,40.874561,-73.909831
